In [ ]:
import geopandas as gpd
from shapely import geometry

In [ ]:
# data_file = "/Users/alex/Data/bathymetry/fiji_4326.gpkg"
data_file = "/Users/alex/Data/bathymetry/TV.gpkg"
data = gpd.read_file(data_file)

# If a value is positive, make it negative
data['depth'] = data['depth'].apply(lambda x: x if x < 0 else -x)

data

In [ ]:
ll = (-8.67780,179.01156)
ur = (-8.39978,179.23369)

subset = data.cx[ll[1]:ur[1], ll[0]:ur[0]]

In [ ]:
crs = 3832
data_projected = subset.to_crs(epsg=crs)

# Get the extent of the shapefile
total_bounds = data_projected.total_bounds

# Get minX, minY, maxX, maxY
minX, minY, maxX, maxY = total_bounds

# Create a fishnet
x, y = (minX, minY)
geom_array = []

# Polygon Size
square_size = 20
while y <= maxY:
    while x <= maxX:
        geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
        geom_array.append(geom)
        x += square_size
    x = minX
    y += square_size

fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs(epsg=crs)

print(f"Created fishnet with {len(fishnet)} polygons")

In [ ]:
# Combine the fishnet with the bathymetry and get the average depth
joined = fishnet.sjoin(data_projected, how="right", predicate="intersects")
print(f"Joined has {len(joined)} rows")

non_null = joined[joined['depth'].notna()]
print(f"Non-null has {len(non_null)} rows")

# Get the average depth and the geometry
average_depth = non_null.groupby('index_left')['depth'].mean()
print(f"Average depth has {len(average_depth)} rows")

# Join index_left to the fishnet
fishnet['index_left'] = fishnet.index
fishnet = fishnet.set_index('index_left')
joined_fishnet = fishnet.join(average_depth, how='inner')

joined_fishnet["geometry"] = joined_fishnet["geometry"].centroid
print(f"Final fishnet has {len(joined_fishnet)} rows")

In [ ]:
# joined_fishnet.explore()

# Get a random sample of points
sample = joined_fishnet.sample(100)

In [ ]:
joined_fishnet.to_file("random_forest_regressor/tuvalu_20.gpkg", overwrite=True)

print(f"Wrote {len(average_depth)} points")